In [1]:
import pickle
from collections import OrderedDict

import numpy as np
from periodictable import elements

In [2]:
from exfor_tools import curate, quantities, reaction
from jitr import rmatrix
from jitr.utils import kinematics
from jitr.xs.elastic import DifferentialWorkspace

import elm

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
with open("nn_elastic_data.pkl", "rb") as f:
    nn_data = pickle.load(f)

In [4]:
with open("pp_elastic_data.pkl", "rb") as f:
    pp_data = pickle.load(f)

In [5]:
all_data = list(pp_data.values()) + list(nn_data.values())
all_data_by_entry, sys_uncertainties_by_entry = (
    curate.cross_reference_entry_systematic_err(all_data)
)

In [6]:
for entry_id, entries in all_data_by_entry.items():
    for entry in entries:
        for measurement in entry.measurements:
            if measurement.systematic_offset_err > 0:
                # TODO
                measurement.systematic_offset_err = 0

In [7]:
from IPython.display import Latex, Math, display  # just for fun

targets = sorted(
    list(set(list(nn_data.keys()) + list(pp_data.keys()))), key=lambda x: x[1] + x[0]
)
display(
    Math(
        ", ".join(
            [f"\,^{{{target[0]}}} \\rm{{{elements[target[1]]}}}" for target in targets]
        )
    )
)

<IPython.core.display.Math object>

In [8]:
entry = pp_data[(40, 20)].data["dXS/dA"].entries

In [9]:
measurement_groups = curate.categorize_measurements_by_energy(
    pp_data[(40, 20)].data["dXS/dA"].entries
)

In [10]:
constraints = []
for measurement_group in measurement_groups:
    for m in measurement_group:
        if np.isclose(m.systematic_norm_err, 0):
            m.systematic_norm_err = 0.05

        constraints.append(
            elm.calibration.ReactionDistribution(
                quantity="dXS/dA",
                measurement=m,
            )
        )

In [11]:
angles_vis = np.linspace(0.01, 180, 100)

In [12]:
core_solver = rmatrix.Solver(40)

In [13]:
models = []
for measurement_group in measurement_groups:
    for m in measurement_group:
        models.append(
            elm.ElasticModel(
                quantity="dXS/dA",
                reaction=pp_data[(40, 20)].reaction,
                Elab=m.Einc,
                angles_rad_constraint=m.x * np.pi / 180,
                angles_rad_vis=angles_vis * np.pi / 180,
                core_solver=core_solver,
                lmax=30,
            )
        )

In [14]:
with open("../prior/prior_distribution.pickle", "rb") as f:
    prior = pickle.load(f)

In [15]:
prior_samples = prior.rvs(size=1000)

In [16]:
prior_mean = OrderedDict([(p.name, v) for p, v in zip(elm.params, prior.mean)])

In [17]:
prior_samples.shape

(1000, 15)

In [18]:
prior_samples[0]

array([52.92488473,  5.04352613, 13.17745736,  5.82701589, -1.89057496,
       23.21689405, -0.36365963, 49.88675807, 25.63981923, -0.24144966,
       -0.18899876,  1.19763674,  1.14565836,  0.73921425,  0.7337632 ])

In [19]:
models[0].get_quantity(elm.calculate_diff_xs, prior_mean) / 1000

array([0.58541755, 0.37994781, 0.26281832, 0.20550015, 0.17961598,
       0.15403256, 0.13326031, 0.11189418, 0.10234007, 0.09451437,
       0.0858876 , 0.08103618, 0.07451652, 0.07161538, 0.06735712,
       0.06550115, 0.06158788, 0.05966718, 0.05698793, 0.05523529,
       0.05302955, 0.05178097, 0.05069354, 0.0496348 , 0.04885668])

In [22]:
xs = np.zeros((1000, len(angles_vis)))

In [31]:
for i in range(1000):
    xs[i, :] = (
        models[0].get_quantity_vis(
            elm.calculate_diff_xs,
            OrderedDict([(p.name, v) for p, v in zip(elm.params, prior_samples[i, :])]),
        )
        / 1000
    )

In [24]:
50-68/2

16.0

In [25]:
upper, lower = np.percentile(xs, [16,84], axis=0)

In [26]:
constraints[0].y

array([0.259  , 0.145  , 0.087  , 0.066  , 0.0594 , 0.066  , 0.0555 ,
       0.0408 , 0.029  , 0.0173 , 0.0084 , 0.00471, 0.00456, 0.0082 ,
       0.0128 , 0.0174 , 0.0214 , 0.0207 , 0.0193 , 0.0157 , 0.012  ,
       0.0088 , 0.0072 , 0.0079 , 0.0091 ])

In [27]:
from matplotlib import pyplot as plt

In [28]:
measurement_groups[0][0].y_units

'barns/ster'